In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [25]:
def get_articles_by_person(name):
    # search for certain person's articles
    driver = webdriver.Chrome('./chromedriver')

    try:
        raw_data = []
        
        driver.get("https://suw.biblos.pk.edu.pl/")
        input_element = driver.find_element("name", "query").send_keys(name)
        submit_button = driver.find_element("xpath", "/html/body/form/table/tbody/tr[3]/td[2]/center/table/tbody/tr[1]/td[2]/input")
        driver.execute_script("arguments[0].click();", submit_button);
        
        # element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="resultsBox"]/center/table[1]')))
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located(("xpath", '//*[@id="resultsDiv_16453"]/center/center/table[2]')))

        for row in driver.find_elements('xpath',"//*[starts-with(@id, 'resourceItemTable')]"):
            record = row.get_attribute('innerHTML')
            raw_data.append(record)
            print(BeautifulSoup(record))
        
        return raw_data
        
    finally:
        driver.delete_all_cookies()
        driver.quit()

In [21]:
def prettify(value):
    return ' '.join(value.split())

def parse_html_data(articles_html):
    articles_soup = BeautifulSoup(articles_html,'html.parser')
    # list of "article" classes
    articles = []

    documents = articles_soup.find_all('html')

    for document in documents:
        soup_title = document.find('a', {'title' : ''})    
        soup_authors = document.find_all('a', {'title' : 'Profil w BPP'})
        soup_info = document.find_all('tbody')[1] 
        
        
        # assign special property of authors
        authors = [author.text for author in soup_authors]
        title = prettify(soup_title.text)
        info = prettify(soup_info.text)    
            
        articles.append(title)
        articles.append(authors)
        articles.append(info)
        
                
    return articles

In [26]:
raw_articles = get_articles_by_person("Wojnar")


/var/folders/3n/zpnxvn457fq5mz2mpm4914vc0000gn/T/ipykernel_2553/3067782014.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


<html><body><tbody><tr valign="top"><td align="left" rowspan="2" width="10"><img hspace="8" src="/suw/files/portal_graph/resource_kind/paper_42.png"/></td><td><a href="/search&amp;termId=2&amp;query=%21Korzekwa+%21Joanna" style="color:#000" title="Pokaż prace tego autora">Korzekwa, Joanna</a>; <a href="/search&amp;termId=2&amp;query=%21Skoneczny+%21W%C5%82adys%C5%82aw" style="color:#000" title="Pokaż prace tego autora">Skoneczny, Władysław</a>; <a href="/userHomepage&amp;uId=326&amp;rel=BPP-author" title="Profil w BPP">Wojnar, Leszek</a> <br/><b><a href="/resourceDetailsBPP&amp;rId=25305">Wpływ parametrów elektroosadzania na zmiany nanostruktury warstw Al2O3/WS2 o przeznaczeniu tribologicznym</a></b></td><td align="right" rowspan="2"><input id="resSelectChecbox_25305" name="resSelectChecbox_25305" onchange="onResourceSelected(25305, this.checked)" style="display: none;" type="checkbox"/></td></tr><tr><td align="left"><center><table border="0" cellpadding="0" cellspacing="1" width="96%"

In [27]:
print(parse_html_data(raw_articles))

TypeError: expected string or bytes-like object

In [28]:
print(raw_articles)

['<tbody><tr valign="top"><td rowspan="2" align="left" width="10"><img src="/suw/files/portal_graph/resource_kind/paper_42.png" hspace="8"></td><td><a style="color:#000" href="/search&amp;termId=2&amp;query=%21Korzekwa+%21Joanna" title="Pokaż prace tego autora">Korzekwa, Joanna</a>; <a style="color:#000" href="/search&amp;termId=2&amp;query=%21Skoneczny+%21W%C5%82adys%C5%82aw" title="Pokaż prace tego autora">Skoneczny, Władysław</a>; <a href="/userHomepage&amp;uId=326&amp;rel=BPP-author" title="Profil w BPP">Wojnar, Leszek</a> <br><b><a href="/resourceDetailsBPP&amp;rId=25305">Wpływ parametrów elektroosadzania na zmiany nanostruktury warstw Al2O3/WS2 o przeznaczeniu tribologicznym</a></b></td><td rowspan="2" align="right"><input id="resSelectChecbox_25305" name="resSelectChecbox_25305" type="checkbox" onchange="onResourceSelected(25305, this.checked)" style="display: none;"></td></tr><tr><td align="left"><center><table width="96%" cellpadding="0" cellspacing="1" border="0"><tbody><tr><